# CFNet (Collaborative Filtering Network) 학습

DeepCF 논문(AAAI 2019)의 최종 CFNet 모델을 PyTorch로 구현한 학습 노트북입니다.

## 모델 개요

CFNet은 DMF와 MLP를 결합한 앙상블 모델로, 두 접근법의 장점을 모두 활용합니다.

$$\hat{y}_{ui} = \sigma(W^T([f_{DMF}(r_u, r_i), f_{MLP}(r_u, r_i)]) + b)$$

- **DMF part**: Element-wise product (representation learning)
- **MLP part**: Concatenation (metric learning)
- **Fusion**: Concatenate DMF and MLP outputs

## Pretrain (권장)

논문에서는 DMF와 MLP를 먼저 개별 학습 후 가중치를 CFNet에 로드하는 방식을 권장합니다.
이는 더 나은 초기값을 제공하여 최종 성능을 향상시킵니다.

## 1. 임포트 및 환경 설정

In [1]:
import sys
sys.path.append('..')

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from time import time
import random
import os

# 공통 유틸리티 임포트
from common.data_utils import load_deepcf_data, get_train_matrix
from common.train_utils import get_train_instances, TrainDataset
from common.eval_utils import evaluate_model

# CFNet 모델 명시적 임포트
from cfnet.cfnet_model import CFNet

# 재현성을 위한 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# 디바이스 설정 (CUDA > MPS > CPU)
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_name = f"CUDA ({torch.cuda.get_device_name(0)})"
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    device_name = "MPS (Apple Silicon)"
else:
    device = torch.device('cpu')
    device_name = "CPU"

print(f"Device: {device_name}")

Device: MPS (Apple Silicon)


## 2. 하이퍼파라미터 설정

In [3]:
# 데이터 설정
DATA_PATH = '../datasets/'
DATASET = 'ml-1m'
USE_SAMPLE = True
SAMPLE_USERS = 100

# DMF 구조 (CFNet의 DMF part)
USERLAYERS = [512, 64]  # User tower 레이어 크기
ITEMLAYERS = [1024, 64]  # Item tower 레이어 크기

# MLP 구조 (CFNet의 MLP part)
LAYERS = [512, 256, 128, 64]  # MLP 레이어 크기

# Pretrain 설정 (중요!)
USE_PRETRAIN = True  # True: pretrain 사용 (권장), False: from scratch
DMF_PRETRAIN = '../pretrain/ml-1m-sample100-rl.pth'
MLP_PRETRAIN = '../pretrain/ml-1m-sample100-ml.pth'

# 학습 파라미터
EPOCHS = 20
BATCH_SIZE = 256
NUM_NEG = 4  # Negative sample 개수
LEARNING_RATE = 0.0001  # CFNet은 DMF와 같은 낮은 학습률 사용
LEARNER = 'adam'

# 평가 및 저장
TOP_K = 10
VERBOSE = 1  # 평가 주기 (epoch)
SAVE_MODEL = True
MODEL_DIR = '../pretrain/'

print(f"Dataset: {DATASET} (Sample: {USE_SAMPLE})")
print(f"DMF: User{USERLAYERS} / Item{ITEMLAYERS}")
print(f"MLP: {LAYERS}")
print(f"Pretrain: {USE_PRETRAIN}")
if USE_PRETRAIN:
    print(f"  - DMF: {DMF_PRETRAIN}")
    print(f"  - MLP: {MLP_PRETRAIN}")
print(f"Training: {EPOCHS} epochs, batch={BATCH_SIZE}, neg={NUM_NEG}, lr={LEARNING_RATE}")

Dataset: ml-1m (Sample: True)
DMF: User[512, 64] / Item[1024, 64]
MLP: [512, 256, 128, 64]
Pretrain: True
  - DMF: ../pretrain/ml-1m-sample100-rl.pth
  - MLP: ../pretrain/ml-1m-sample100-ml.pth
Training: 20 epochs, batch=256, neg=4, lr=0.0001


## 3. 데이터 준비

샘플 데이터가 필요한 경우 `../data_sampling.ipynb`를 먼저 실행하세요.

**데이터 포맷:**
- `train.rating`, `test.rating`: `userID\titemID\trating\ttimestamp`
- `test.negative`: `(userID,itemID)\tneg1\tneg2\t...` (99개)

In [4]:
dataset_name = f'{DATASET}-sample{SAMPLE_USERS}' if USE_SAMPLE else DATASET

print(f"Loading dataset: {dataset_name}...")
t1 = time()
train, testRatings, testNegatives, num_users, num_items = load_deepcf_data(
    DATA_PATH, dataset_name
)
print(f"Loaded in {time()-t1:.1f}s")
print(f"  Users: {num_users}, Items: {num_items}")
print(f"  Train: {train.nnz}, Test: {len(testRatings)}")

Loading dataset: ml-1m-sample100...
Loaded in 0.1s
  Users: 100, Items: 2591
  Train: 17361, Test: 93


## 4. 모델 생성

**CFNet 구조:**
- DMF part:
  - User tower: $|I| \rightarrow 512 \rightarrow 64$
  - Item tower: $|U| \rightarrow 1024 \rightarrow 64$
  - DMF vector: element-wise product (64 dim)
- MLP part:
  - User/Item embedding: $|I|, |U| \rightarrow 256$
  - Concat: $[256, 256] \rightarrow 512$
  - MLP: $512 \rightarrow 256 \rightarrow 128 \rightarrow 64$
- Fusion:
  - Concat: $[64_{DMF}, 64_{MLP}] \rightarrow 128$
  - Prediction: $\text{sigmoid}(Linear(128, 1))$

In [5]:
# Train matrix 변환 (sparse -> dense)
train_matrix = get_train_matrix(train)

# 모델 초기화 (Pretrain 또는 From Scratch)
if USE_PRETRAIN:
    print(f"Loading pretrained models...")
    model = CFNet(train_matrix, num_users, num_items,
                  USERLAYERS, ITEMLAYERS, LAYERS,
                  dmf_pretrain_path=DMF_PRETRAIN,
                  mlp_pretrain_path=MLP_PRETRAIN).to(device)
    print(f"✓ Pretrain loaded from:")
    print(f"  - DMF: {DMF_PRETRAIN}")
    print(f"  - MLP: {MLP_PRETRAIN}")
else:
    print(f"Initializing from scratch...")
    model = CFNet(train_matrix, num_users, num_items,
                  USERLAYERS, ITEMLAYERS, LAYERS).to(device)
    print(f"✓ Random initialization (Lecun normal)")

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"\nModel: {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"Optimizer: {LEARNER}, LR: {LEARNING_RATE}")

Loading pretrained models...
✓ Pretrain loaded from:
  - DMF: ../pretrain/ml-1m-sample100-rl.pth
  - MLP: ../pretrain/ml-1m-sample100-ml.pth

Model: 2,390,977 parameters
Optimizer: adam, LR: 0.0001


## 5. 초기 성능 평가

**평가 메트릭:**
- Hit Ratio@K: Top-K 내 정답 포함 비율
- NDCG@K: Normalized Discounted Cumulative Gain

**Pretrain vs From Scratch:**
- Pretrain 사용 시: 초기 성능이 DMF, MLP보다 높음 (가중치 물려받음)
- From scratch: 초기 성능이 매우 낮음 (랜덤 초기화)

In [6]:
print("Evaluating initial performance...")
t1 = time()
hits, ndcgs = evaluate_model(model, testRatings, testNegatives, TOP_K, device)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print(f"Init: HR@{TOP_K}={hr:.4f}, NDCG@{TOP_K}={ndcg:.4f} [{time()-t1:.1f}s]")

if USE_PRETRAIN:
    print(f"\n✓ Pretrain 효과: 초기 성능이 DMF/MLP보다 높아야 함")
else:
    print(f"\n✓ From scratch: 초기 성능이 낮음 (랜덤 초기화)")

best_hr, best_ndcg, best_iter = hr, ndcg, -1

Evaluating initial performance...
Init: HR@10=0.4086, NDCG@10=0.2346 [0.5s]

✓ Pretrain 효과: 초기 성능이 DMF/MLP보다 높아야 함


## 6. 학습

**학습 과정:**
1. Negative sampling: 각 positive에 대해 $N$ 개의 negative 샘플 생성
2. Mini-batch SGD로 BCE loss 최소화
   $$\mathcal{L} = -\sum_{(u,i)} y_{ui} \log \hat{y}_{ui} + (1-y_{ui}) \log(1-\hat{y}_{ui})$$
3. 매 epoch 평가 및 best model 저장

**예상 성능 (논문 기준):**
- DMF: HR@10 ≈ 0.68
- MLP: HR@10 ≈ 0.69
- **CFNet (pretrain)**: HR@10 ≈ 0.70 ⭐ (최고 성능)

In [7]:
# CFNet은 DMF + MLP fusion 모델이므로 별도로 저장하지 않음
# (DMF-rl.pth와 MLP-ml.pth만 있으면 CFNet 재구성 가능)
print("\nStarting training...")
print("=" * 80)

for epoch in range(EPOCHS):
    t1 = time()
    
    # Negative sampling
    user_input, item_input, labels = get_train_instances(train, NUM_NEG, num_items)
    train_dataset = TrainDataset(user_input, item_input, labels)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # Training
    model.train()
    total_loss = 0
    for batch_users, batch_items, batch_labels in train_loader:
        batch_users = batch_users.to(device)
        batch_items = batch_items.to(device)
        batch_labels = batch_labels.to(device).unsqueeze(1)
        
        predictions = model(batch_users, batch_items)
        loss = criterion(predictions, batch_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    t2 = time()
    
    # Evaluation
    if epoch % VERBOSE == 0:
        hits, ndcgs = evaluate_model(model, testRatings, testNegatives, TOP_K, device)
        hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
        
        print(f'Epoch {epoch:2d} [{t2-t1:.1f}s]: HR={hr:.4f}, NDCG={ndcg:.4f}, '
              f'loss={avg_loss:.4f} [{time()-t2:.1f}s]')
        
        # Best performance tracking (저장하지 않음)
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            print(f'  → Best performance updated (HR={hr:.4f})')

print("=" * 80)
print("Training completed!")


Starting training...
Epoch  0 [3.0s]: HR=0.4516, NDCG=0.2183, loss=0.0122 [0.1s]
  → Best performance updated (HR=0.4516)
Epoch  1 [1.3s]: HR=0.4409, NDCG=0.2030, loss=0.0074 [0.1s]
Epoch  2 [1.3s]: HR=0.4516, NDCG=0.2125, loss=0.0057 [0.1s]
Epoch  3 [1.3s]: HR=0.4409, NDCG=0.2050, loss=0.0054 [0.1s]
Epoch  4 [1.3s]: HR=0.4301, NDCG=0.2021, loss=0.0043 [0.1s]
Epoch  5 [1.2s]: HR=0.4194, NDCG=0.1993, loss=0.0040 [0.1s]
Epoch  6 [1.3s]: HR=0.4301, NDCG=0.2115, loss=0.0039 [0.1s]
Epoch  7 [1.2s]: HR=0.4194, NDCG=0.2026, loss=0.0036 [0.1s]
Epoch  8 [1.2s]: HR=0.4409, NDCG=0.2083, loss=0.0026 [0.1s]
Epoch  9 [1.2s]: HR=0.3763, NDCG=0.1893, loss=0.0025 [0.1s]
Epoch 10 [1.2s]: HR=0.4301, NDCG=0.2052, loss=0.0025 [0.1s]
Epoch 11 [1.2s]: HR=0.3978, NDCG=0.1936, loss=0.0021 [0.1s]
Epoch 12 [1.2s]: HR=0.3978, NDCG=0.1947, loss=0.0020 [0.1s]
Epoch 13 [1.2s]: HR=0.4086, NDCG=0.1891, loss=0.0017 [0.1s]
Epoch 14 [1.2s]: HR=0.4301, NDCG=0.1996, loss=0.0015 [0.1s]
Epoch 15 [1.2s]: HR=0.3978, NDCG=0.20

## 7. 결과

In [8]:
print("\n" + "=" * 80)
print("FINAL RESULTS")
print("=" * 80)
print(f"Best Epoch: {best_iter}")
print(f"Best HR@{TOP_K}: {best_hr:.4f}")
print(f"Best NDCG@{TOP_K}: {best_ndcg:.4f}")

print("\nConfiguration:")
print(f"  Dataset: {dataset_name}")
print(f"  DMF: User{USERLAYERS}, Item{ITEMLAYERS}")
print(f"  MLP: {LAYERS}")
print(f"  Pretrain: {USE_PRETRAIN}")
print(f"  Training: {EPOCHS} epochs, batch={BATCH_SIZE}, neg={NUM_NEG}")
print(f"  Optimizer: {LEARNER}, lr={LEARNING_RATE}")

print("\n" + "=" * 80)
print("성능 비교 (참고):")
print("  DMF (sample-100): HR@10 ≈ 0.79")
print("  MLP (sample-100): HR@10 ≈ 0.72")
print(f"  CFNet (현재):    HR@10 = {best_hr:.4f}")
print("\n✓ CFNet이 DMF, MLP보다 높은 성능을 보이면 성공!")
print("\nNote: CFNet은 별도로 저장하지 않습니다.")
print("      (DMF와 MLP pretrain 가중치만 있으면 CFNet 재구성 가능)")
print("=" * 80)


FINAL RESULTS
Best Epoch: 0
Best HR@10: 0.4516
Best NDCG@10: 0.2183

Configuration:
  Dataset: ml-1m-sample100
  DMF: User[512, 64], Item[1024, 64]
  MLP: [512, 256, 128, 64]
  Pretrain: True
  Training: 20 epochs, batch=256, neg=4
  Optimizer: adam, lr=0.0001

성능 비교 (참고):
  DMF (sample-100): HR@10 ≈ 0.79
  MLP (sample-100): HR@10 ≈ 0.72
  CFNet (현재):    HR@10 = 0.4516

✓ CFNet이 DMF, MLP보다 높은 성능을 보이면 성공!

Note: CFNet은 별도로 저장하지 않습니다.
      (DMF와 MLP pretrain 가중치만 있으면 CFNet 재구성 가능)
